<h1>Sentiment Analysis<h1>


<img src = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRCyAiLX3F1VpZtSX2VT8yNmsJU0vWSgUq7vmvhtyFCqQ&usqp=CAU&ec=48600112'
     height = "600"
     width = "700"/>

# Index: <a class="anchor" id="index"></a>
* [1. Library Imports](#imports)
* [2. Function Defination](#functions)
* [3. Model Defination](#model-defination)
* [4. Data Preprocessing](#data-preprocessing)
* [5. Sentiment Analysis](#sentiment-analysis)

# Library Imports <a class="anchor" id="imports"></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.7 MB/s eta 0:00:00


In [ ]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import datetime
import os.path
import re
import time
from datetime import datetime
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import logging
import json
from datetime import datetime


* [Go to Top](#index)

# Function Defination <a class="anchor" id="functions"></a>

In [ ]:
def find_sentiment(tweet):
    encoded_input = tokenizer(tweet, return_tensors='pt')
    output = model(**encoded_input)
    scores = output.logits.detach().numpy()[0]
    return scores
   

* [Go to Top](#index)

# Model Defination <a class="anchor" id="model-defination"></a>

In [ ]:
model_name = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

* [Go to Top](#index)

# Data Preprocessing <a class="anchor" id="data-preprocessing"></a>

In [ ]:
file_list=["/content/Chewy_2023-04-02 182508.csv"]  #Change this to the required file location or file name


In [ ]:
for file_name in file_list:
    # read the file into a dataframe
    df_full_file = pd.read_csv(file_name)

    # apply the code to the dataframe
    df_new = pd.DataFrame()
    df_old = pd.DataFrame()
    if 'Sentiment' in df_full_file.columns:
        mask = df_full_file['Sentiment'].isnull()
        df_new = df_full_file[mask]
        df_old = df_full_file[~mask]

        df = pd.DataFrame(columns=df_full_file.columns)
        for index, row in df_new.iterrows():
            tweet_id = row['Id']
            if tweet_id in df_old['Id'].values:
                # Update retweet and like count
                df_old.loc[df_old['Id'] == tweet_id, 'Retweet Count'] = row['Retweet Count'] 
                df_old.loc[df_old['Id'] == tweet_id, 'Favorite Count'] = row['Favorite Count']
                df_old.loc[df_old['Id'] == tweet_id, 'Total Tweets Count'] = int(row['Retweet Count'] + 1)
            else:
                df = pd.concat([df, row.to_frame().T])    
    else:
        df = df_full_file

    # write the modified dataframe back to the file
    df.to_csv(file_name, index=False)


* [Go to Top](#index)

# Sentiment Analysis <a class="anchor" id="sentiment-analysis"></a>

In [ ]:
for file in file_list:
    # read the file into a dataframe
    df = pd.read_csv(file)

    # apply sentiment analysis to the tweets in the dataframe
    start_time = time.time()
    df["Sentiment Score"] = df["Tweet"].apply(lambda tweet: find_sentiment(tweet))
    df["Sentiment"] = df["Sentiment Score"].apply(lambda score: int(np.argmax(score)))
    end_time = time.time()
    print("Time required for sentiment analysis: ", end_time-start_time)

    # replace numerical sentiment labels with corresponding string values
    dict1 = {0:"Negative",1:"Neutral",2:"Positive"}
    df.replace({"Sentiment":dict1},inplace=True)

    # create columns for positive, neutral, and negative sentiment scores
    df["Positive Sentiment Score"] = df["Sentiment Score"].apply(lambda sentiment: sentiment[2])
    df["Neutral Sentiment Score"] = df["Sentiment Score"].apply(lambda sentiment: sentiment[1])
    df["Negative Sentiment Score"] = df["Sentiment Score"].apply(lambda sentiment: sentiment[0])

    # write the modified dataframe back to the file
    df.to_csv(file, index=False)  #Change this to the required file location

Time required for sentiment analysis:  10.690197467803955


* [Go to Top](#index)